In [1]:
!pip install simpy plotly

In [2]:
import simpy

In [3]:
# 

In [4]:
class SystemResource(simpy.Resource):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        print(args)
        print(kwargs)
        print(f"create resource with capacity {self.capacity}")
        self.processing_data = []
        self.waiting_data = []

    def request(self, *args, **kwargs):
        print("request resource at %d" % self._env.now)
        # start with len(self.queue as documented
        print(f"{len(self.queue)} waiting at %d" % self._env.now)
        # but only shows queue
        # attention users et pas resources
        # attention compteur mesuré a avant (donc 0)
        print(f"{self.count} processing at %d" % self._env.now)
        self.processing_data.append((self._env.now, self.count))
        self.waiting_data.append((self._env.now, len(self.queue)))
        return super().request(*args, **kwargs)

    def release(self, *args, **kwargs):
        print("release resource at %d" % self._env.now)
        print(f"{len(self.queue)} waiting at %d" % self._env.now)
        # attention compteur mesuré a avant (donc 1)
        print(f"{self.count} processing at %d" % self._env.now)
        self.processing_data.append((self._env.now, self.count))
        self.waiting_data.append((self._env.now, len(self.queue)))
        return super().release(*args, **kwargs)



In [5]:
class User():
    def __init__(self, id, env, res, monitor):
        self.id = id
        self.env = env
        self.res = res
        self.monitor = monitor
        # ajoute apres sinon chevauchement
        self.taskid = 1
        self.name = f"User-{id}"
        # Start the run process everytime an instance is created.
        # create itself as a processs
        self.action = env.process(self.run())

    def run(self):
        while True:
            taskname = "process_task"
            self.taskid += 1
            self.monitor.report_start(self.name, taskname, self.taskid)
            print(f"{self.name} starts task at %d" % self.env.now)
            task_duration = 5
            # We yield the process that process() returns
            # to wait for it to finish
            yield self.env.process(self.process_task(task_duration))
            print(f"{self.name} ends task at %d" % self.env.now)
            self.monitor.report_stop(self.name, taskname, self.taskid)
     
            # The charge process has finished and
            # we can start driving again.
            print(f"{self.name} starts verification at %d" % self.env.now)
            verification_duration = 2
            yield self.env.timeout(verification_duration)
            print(f"{self.name} ends verification at %d" % self.env.now)

    def process_task(self, duration):
        with self.res.request() as req:
            yield req
            yield self.env.timeout(duration)


In [6]:
import datetime
class Clock:
    def __init__(self):
        #self.base_epoch = datetime.datetime(2021,9,10,0,0).timestamp()
        self.base_epoch = datetime.datetime.now().timestamp()
        print(f"Clock created - base {self.base_epoch}")

    def to_date(self, tick):
        epoch_time = self.base_epoch + tick*60 #mn
        #print(f"Clock {epoch_time}")
        datetime_time = datetime.datetime.fromtimestamp(epoch_time)
        return datetime_time

In [7]:
clock = Clock()

Clock created - base 1631380468.956452


In [8]:
import pandas as pd
class UsersMonitor:
    def __init__(self, env, clock):
        self.env = env
        self.start_data = []
        self.stop_data = []
        
    def report_start(self, username, taskname, taskid):
        mark = self.env.now
        self.start_data.append(
            dict(  
                StartMark=mark,
                Start=clock.to_date(mark),
                Username=username,
                Task=taskname,
                TaskId=taskid
            )
        )       
        
    def report_stop(self, username, taskname, taskid):
        mark = self.env.now
        self.stop_data.append(
            dict(  
                FinishMark=mark,
                Finish=clock.to_date(mark),
                Username=username,
                Task=taskname,
                TaskId=taskid
            )
        )
        
                
    def collect(self):
        df_start = pd.DataFrame(self.start_data)
        df_stop = pd.DataFrame(self.stop_data)
        df = pd.merge(df_start, df_stop, how='left', 
                      on = ['Username', 'Task', 'TaskId'])
        return df


In [9]:
env = simpy.Environment()

user_monitor = UsersMonitor(env, clock)
res = SystemResource(env, capacity=1)
user = User(1, env, res, user_monitor)
env.run(until=15)

print("Processing")
print(res.processing_data)
print("Waiting")
print(res.waiting_data)

(<simpy.core.Environment object at 0x10c424b70>,)
{'capacity': 1}
create resource with capacity 1
User-1 starts task at 0
request resource at 0
0 waiting at 0
0 processing at 0
release resource at 5
0 waiting at 5
1 processing at 5
User-1 ends task at 5
User-1 starts verification at 5
User-1 ends verification at 7
User-1 starts task at 7
request resource at 7
0 waiting at 7
0 processing at 7
release resource at 12
0 waiting at 12
1 processing at 12
User-1 ends task at 12
User-1 starts verification at 12
User-1 ends verification at 14
User-1 starts task at 14
request resource at 14
0 waiting at 14
0 processing at 14
Processing
[(0, 0), (5, 1), (7, 0), (12, 1), (14, 0)]
Waiting
[(0, 0), (5, 0), (7, 0), (12, 0), (14, 0)]


In [10]:
def user_generator(size, env, res, user_monitor):
    for i in range(size):
        User(i, env, res, user_monitor).action
        yield env.timeout(5)

In [11]:
# au passage suppression de magix numbers
sim_duration = 100
nb_users = 20

env = simpy.Environment()
user_monitor = UsersMonitor(env, clock)
res = SystemResource(env, capacity=1)
#user = User(env, res)
user_gen = env.process(user_generator(nb_users, env, res, user_monitor))

env.run(until=sim_duration)



(<simpy.core.Environment object at 0x10c432fd0>,)
{'capacity': 1}
create resource with capacity 1
User-0 starts task at 0
request resource at 0
0 waiting at 0
0 processing at 0
User-1 starts task at 5
request resource at 5
0 waiting at 5
1 processing at 5
release resource at 5
1 waiting at 5
1 processing at 5
User-0 ends task at 5
User-0 starts verification at 5
User-0 ends verification at 7
User-0 starts task at 7
request resource at 7
0 waiting at 7
1 processing at 7
User-2 starts task at 10
request resource at 10
1 waiting at 10
1 processing at 10
release resource at 10
2 waiting at 10
1 processing at 10
User-1 ends task at 10
User-1 starts verification at 10
User-1 ends verification at 12
User-1 starts task at 12
request resource at 12
1 waiting at 12
1 processing at 12
User-3 starts task at 15
request resource at 15
2 waiting at 15
1 processing at 15
release resource at 15
3 waiting at 15
1 processing at 15
User-0 ends task at 15
User-0 starts verification at 15
User-0 ends verifi

In [12]:
print("Processing")
print(res.processing_data)


Processing
[(0, 0), (5, 1), (5, 1), (7, 1), (10, 1), (10, 1), (12, 1), (15, 1), (15, 1), (17, 1), (20, 1), (20, 1), (22, 1), (25, 1), (25, 1), (27, 1), (30, 1), (30, 1), (32, 1), (35, 1), (35, 1), (37, 1), (40, 1), (40, 1), (42, 1), (45, 1), (45, 1), (47, 1), (50, 1), (50, 1), (52, 1), (55, 1), (55, 1), (57, 1), (60, 1), (60, 1), (62, 1), (65, 1), (65, 1), (67, 1), (70, 1), (70, 1), (72, 1), (75, 1), (75, 1), (77, 1), (80, 1), (80, 1), (82, 1), (85, 1), (85, 1), (87, 1), (90, 1), (90, 1), (92, 1), (95, 1), (95, 1), (97, 1)]


In [13]:
print("Waiting")
print(res.waiting_data)

Waiting
[(0, 0), (5, 0), (5, 1), (7, 0), (10, 1), (10, 2), (12, 1), (15, 2), (15, 3), (17, 2), (20, 3), (20, 4), (22, 3), (25, 4), (25, 5), (27, 4), (30, 5), (30, 6), (32, 5), (35, 6), (35, 7), (37, 6), (40, 7), (40, 8), (42, 7), (45, 8), (45, 9), (47, 8), (50, 9), (50, 10), (52, 9), (55, 10), (55, 11), (57, 10), (60, 11), (60, 12), (62, 11), (65, 12), (65, 13), (67, 12), (70, 13), (70, 14), (72, 13), (75, 14), (75, 15), (77, 14), (80, 15), (80, 16), (82, 15), (85, 16), (85, 17), (87, 16), (90, 17), (90, 18), (92, 17), (95, 18), (95, 19), (97, 18)]
